<a href="https://colab.research.google.com/github/celikn/GEE_Repository/blob/main/GEE_Image_Export_AOI_20220222.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install earthengine-api
!pip install earthengine-api --upgrade

!pip install geetools
!pip install --upgrade geetools
!pip install geemap

     |████████████████████████████████| 236 kB 12.5 MB/s 
  Created wheel for earthengine-api: filename=earthengine_api-0.1.300-py3-none-any.whl size=265522 sha256=56083178b3a83a9f37495dbf118b683f63db65bd69acc026c953a290044c50fd
  Stored in directory: /root/.cache/pip/wheels/f1/44/b9/ec79598e6147ec21b1dcbd669b963afcdd1e9326b48127fd85
Successfully built earthengine-api
  Attempting uninstall: earthengine-api
    Found existing installation: earthengine-api 0.1.299
    Uninstalling earthengine-api-0.1.299:
      Successfully uninstalled earthengine-api-0.1.299
     |████████████████████████████████| 74 kB 2.9 MB/s 
     |████████████████████████████████| 44 kB 2.7 MB/s 
  Created wheel for geetools: filename=geetools-0.6.14-py3-none-any.whl size=92107 sha256=6e85ce5e73c6307ce9f88029427efb447a3b2b0d0b44e9dce8ec0857a9c3cf5e
  Stored in directory: /root/.cache/pip/wheels/5c/55/29/0a09dcf6b39bba8c890e40171cac3e3607a5c4b354b7e9447c
Successfully built geetools
     |███████████████████████████

In [ ]:
import ee
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

In [4]:
import geemap
import json
import os
import requests
from geemap import geojson_to_ee, ee_to_geojson
from ipyleaflet import GeoJSON

In [6]:
file_path = os.path.abspath('/content/drive/MyDrive/AnkaraMahalleler.geojson')
with open(file_path) as f:
    json_data = json.load(f)
    print (json_data)

{'type': 'FeatureCollection', 'features': [{'geometry': {'coordinates': [[[32.66558686716756, 39.78868814216332], [32.668052809375844, 39.7884725142255], [32.669701191968194, 39.78859829878445], [32.66960239838077, 39.78763709837098], [32.66984938217096, 39.786671391273636], [32.67069834669936, 39.785499081124605], [32.672283843280205, 39.78401234640966], [32.676106192695705, 39.78152401610453], [32.67676641087531, 39.78016309363173], [32.67717965926069, 39.778986283510775], [32.67769617429968, 39.778424829919544], [32.67845973599038, 39.77813291676975], [32.67933112722727, 39.77809245287712], [32.6808223622315, 39.77857758213614], [32.68321182713863, 39.77998346280877], [32.68443804293345, 39.77981274180678], [32.68647722126645, 39.779669047994894], [32.688592724606494, 39.77960166898425], [32.68994919572909, 39.780010370250395], [32.69130116704963, 39.7801765824972], [32.69210065412178, 39.78003283248253], [32.693416743110284, 39.78056281381056], [32.694862956291246, 39.7800283962300

In [7]:
mahalleler=ee.FeatureCollection(json_data)
aoi_geometry = mahalleler.geometry()


In [8]:
s2= ee.ImageCollection('COPERNICUS/S2')\
.filterDate("2020-11-01","2020-12-31")\
.filterBounds(aoi_geometry)\
.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 5))\
.map(lambda image: image.clip(aoi_geometry.bounds())) ## tum imageleri clip yapalim. 


 ## .select(["B2", "B3", "B4", "B5", "B6", "B7", "B8", "B8A", "B11", "B12"])
## CollectionInfo
images_info= s2.aggregate_array('system:index').getInfo()
print(images_info)
print("image sayisi:"+str(len(images_info)))




['20201102T084119_20201102T084124_T36TVK', '20201117T084241_20201117T084312_T36TVK', '20201127T084311_20201127T084312_T36TVK', '20201202T084329_20201202T084324_T36TVK', '20201207T084341_20201207T084335_T36TVK']
image sayisi:5


In [9]:
json_layer = GeoJSON(
    data=json_data,
    name='Ankara Mahalleler JSON',
    hover_style={'fillColor': 'red', 'fillOpacity': 0.5},
)
lon,lat =32.717443, 39.824942 ## Ankara
Map = geemap.Map(center=[lat,lon], zoom=14) ## Haritayi olusturur. 
## Harita da gormek icin rgb bandlarini visualize ediyoruz. 
rgb = ['B4','B3','B2']
rgbViz = {"min":0.0, "max":3000,"bands":rgb}
firstImageClipped = s2.first().clip(aoi_geometry)
Map.centerObject(aoi_geometry,12)
Map.addLayer(mahalleler, {}, 'Mahalleler');
Map.addLayer(firstImageClipped, rgbViz, "S2")
Map.add_layer(json_layer)
Map


Map(center=[39.86462429494558, 32.69548924623127], controls=(WidgetControl(options=['position', 'transparent_b…

In [ ]:
############# Calisan Ornek Sentinel Imageleri Tiff olarak Google Drive'a indirme ################################
## Read More : https://github.com/gee-community/gee_tools
import geetools
images_info= s2.aggregate_array('system:index').getInfo()
print(images_info)
print("image sayisi:"+str(len(images_info)))

data_type = 'float32'
name_pattern = 'S2SR_{system_date}'
date_pattern = 'yMMdd' # dd: day, MMM: month (JAN), y: year
scale = 10
folder_name = 'SentinelImages_Ankara_Clipped'

tasks = geetools.batch.Export.imagecollection.toDrive(
            collection=s2,
            folder=folder_name,
            namePattern=name_pattern,
            scale=scale,
            datePattern=date_pattern,
            verbose=True,
            maxPixels=1e10)

['20201102T084119_20201102T084124_T36TVK', '20201117T084241_20201117T084312_T36TVK', '20201127T084311_20201127T084312_T36TVK', '20201202T084329_20201202T084324_T36TVK', '20201207T084341_20201207T084335_T36TVK']
image sayisi:5
exporting S2SR_20201102 to folder 'SentinelImages_Ankara_Clipped' in GDrive
exporting S2SR_20201117 to folder 'SentinelImages_Ankara_Clipped' in GDrive
exporting S2SR_20201127 to folder 'SentinelImages_Ankara_Clipped' in GDrive
exporting S2SR_20201202 to folder 'SentinelImages_Ankara_Clipped' in GDrive
exporting S2SR_20201207 to folder 'SentinelImages_Ankara_Clipped' in GDrive
